In [ ]:
import json
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import datetime
import math
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
from pprint import pprint
from copy import deepcopy
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans, Birch, DBSCAN, SpectralClustering, OPTICS, AgglomerativeClustering, SpectralClustering
from keras.models import Model, Sequential
from keras.layers import Dense
from keras import layers, losses
from keras.utils import Progbar
from keras import backend as K
from MixedAutoencoder import Mixer, MixedAutoencoder, setRandom
seed = 12345
tf.random.set_seed(seed)
random.seed(seed)
setRandom(seed)
import json
# important variable
latent_dim = 3
num_sets = 4
base_path = "."
label = "demo4_3_test_1"
model_shape = [16]

In [ ]:
data = pd.read_csv(f'{base_path}/data/16PF/data.csv', sep="\t")
data.head()

In [ ]:
def clean_data(data):
    #remove unwanted columns
    data.pop("age")
    data.pop("gender")
    data.pop("accuracy")
    data.pop("country")
    data.pop("source")
    data.pop("elapsed")
    data.pop("P10")

    #replace zeros with three, questions not answered replaced with average value
    return (data.loc[(data!=0).all(axis=1)]-3)
    #data.replace(0,3, inplace=True)

In [ ]:
def split(data, split_labels = None):
    if not split_labels:
        labels = list(data.columns)
        random.shuffle(labels)
        split_labels = {"set1": labels[:50], "set2": labels[50:100], "set3": labels[100:150]}
    return {k: data[s] for k, s in split_labels.items}


In [ ]:
def split_strat(data, num_sets = 3):
    # general idea is to split the data by question type letter(A), and then randomly select 1/3 of each letter to form
    # three mixed data sets

    # creating a dict of data frames for each letter in ASCII
    framesDict = {}
    for i in range(65, 81):
        framesDict[i] = pd.DataFrame()

    # running through data and sorting it into data frame based on letter
    for i in range(65, 81):
        for columns in data:
            if chr(i) in columns:
                framesDict[i][columns] = data[[columns]]
        

    # creating a dict containing three different data frames for eat data "set"

    setsDict = []
    #set_list = ['set1', 'set2', 'set3']
    set_list = list(range(num_sets))
    for s in set_list:
        setsDict.append(pd.DataFrame())


    # test code: works and prints the data set containing all A questions/answers
    #print (framesDict[65])

    # shuffles each dataframe so it's randomized when columns are later selected
    for i in range (65, 81):
        framesDict[i] = framesDict[i].sample(frac=1, axis=1)

    # idea here was to use iloc to select values in a range from variables a - b
    # and then increment a and b each time the "set" increments so that the data in each frame is split into the three sets
    # not working tho lol


    # calculate how many columns per "set" based on the amount of number of that letter question type
    # cols = len(framesDict[65].axes[1])
    # increment = int((cols - cols%3) /3) 


    # go through each data frame, resetting a and b each time
    remainders = pd.DataFrame()
    for i in range (65, 81):
        cols = len(framesDict[i].axes[1])
        increment = int((cols - cols%num_sets) /num_sets) 
        remainders = framesDict[i].iloc[:,(cols//num_sets) * num_sets:].join(remainders)
        a = 0
        b= increment 
        for s in set_list:   
            # print(setsDict[s])                      # go through each of the three sets
            # print(framesDict[i].iloc[:,a:b])
            
            setsDict[s] = framesDict[i].iloc[:,a:b].join(setsDict[s]  )     # and add on values from range a-b in the current letter/question dataframe
            # print(setsDict[s])  
            a += increment                                  # ^^^^ this line is probably the problem                        
            b += increment                                  # then increment a and b before moving on to next set



    cols = len(remainders.axes[1])
    increment = int((cols - cols%num_sets) /num_sets) 
    a = 0
    b= increment 
    for s in set_list:   
        # print(setsDict[s])                      # go through each of the three sets
        # print(framesDict[i].iloc[:,a:b])
        
        setsDict[s] = remainders.iloc[:,a:b].join(setsDict[s]  )     # and add on values from range a-b in the current letter/question dataframe
        # print(setsDict[s])  
        a += increment                                  # ^^^^ this line is probably the problem                        
        b += increment                                  # then increment a and b before moving on to next set



    return setsDict#['set1'], setsDict['set2'], setsDict['set3']]

In [ ]:
def split_by_col_l(data, cols):
    new_data = []
    for col in cols:
        new_data.append(data[col])

    return new_data

In [ ]:
def make_train_test(datas, split):
    train_datas = [d[:int(datas.shape[0]*split)] for d in datas]
    test_datas = [d[int(datas.shape[0]*split):] for d in datas]
    return train_datas, test_datas

In [ ]:
data = clean_data(data)
#labels = list(data.columns)
#random.shuffle(labels)
#split_labels = [labels[:50], labels[50:100], labels[100:150]]
#print(data.columns[:])
#datas = split(data, split_labels = split_labels)
datas = split_strat(data, num_sets)
#cols = [['P4', 'O6', 'O10', 'N10', 'N8', 'M3', 'M9', 'P5', 'P1', 'O9', 'O3', 'N5', 'N6', 'M8', 'M2', 'L8', 'L6', 'K10', 'K1', 'J5', 'J9', 'I1', 'I7', 'H3', 'H8', 'G4', 'G3', 'F5', 'F10', 'E1', 'E8', 'D3', 'D10', 'C4', 'C9', 'B7', 'B6', 'B2', 'A1', 'A6'],['L5', 'L4', 'K3', 'K8', 'J3', 'J8', 'I4', 'P7', 'P3', 'O2', 'O5', 'N7', 'N3', 'M4', 'M10', 'L9', 'L2', 'K4', 'K9', 'J1', 'J10', 'I6', 'I8', 'H5', 'H6', 'G8', 'G1', 'F6', 'F1', 'E7', 'E2', 'D8', 'D2', 'C2', 'C3', 'B5', 'B12', 'B8', 'A3', 'A9'],['I10', 'H4', 'H10', 'G5', 'G6', 'F9', 'F4', 'P6', 'P2', 'O8', 'O4', 'N2', 'N1', 'M7', 'M6', 'L10', 'L3', 'K7', 'K6', 'J6', 'J7', 'I3', 'I2', 'H2', 'H1', 'G10', 'G7', 'F8', 'F3', 'E4', 'E5', 'D9', 'D5', 'C7', 'C10', 'B4', 'B1', 'B13', 'A2', 'A7'],['E3', 'E9', 'D7', 'D6', 'C8', 'C5', 'B3', 'P9', 'P8', 'O7', 'O1', 'N9', 'N4', 'M5', 'M1', 'L7', 'L1', 'K5', 'K2', 'J4', 'J2', 'I9', 'I5', 'H9', 'H7', 'G9', 'G2', 'F7', 'F2', 'E6', 'E10', 'D1', 'D4', 'C6', 'C1', 'B9', 'B11', 'B10', 'A8', 'A10']]
#datas = split_by_col_l(data, cols)
datas[0].head()

In [ ]:
#Input Output Interface

def interface(Qlist):
    f = open(f'{base_path}/data/16PF/16.PF.questions.csv')  
    print(f)
    #with open ("16.PF.questions.csv") as f:                          (what I was originally using in my own file)
    Qdict = {}
    for line in f.readlines():
        line = line.split(',',1)
        if line:
            Qdict[line[0]]=line[1]
    print(Qdict.keys())

    print("Please answer the following questions on a scale from 1-5, with 1 being strongly disagree, and 5 being strongly agree")

    new = {}
    # ask the question corresponding to the letter code, and add the user input to a dict
    for x in range (0, len(Qlist)):
        new[Qlist[x]] = int(input(Qdict[Qlist[x]].split("rated", 1)[0]))

    # add the dictionary with questions and answers to the dataframe
    responses = pd.DataFrame()
    responses = responses.append(new, ignore_index=True)

    #print(responses)
    return responses

In [ ]:
def present(Qlist, ans):
    f = open(f'{base_path}/data/16PF/16.PF.questions.csv')  
    #print(f)
    #with open ("16.PF.questions.csv") as f:                          (what I was originally using in my own file)
    Qdict = {}
    for line in f.readlines():
        line = line.split(',',1)
        if line:
            Qdict[line[0]]=line[1]

    print(Qdict.keys())

    new = {}
    # ask the question corresponding to the letter code, and add the user input to a dict
    for x in range (0, len(Qlist)):
        print(Qdict[Qlist[x]].split("rated", 1)[0] + ": " + str(ans[0][x]))


In [ ]:
Qlist = ['A1', 'A2', 'A3', 'A4']
# interface(Qlist)

In [ ]:
datas[0].shape

In [ ]:
int(datas[0].shape[0]*0.8)

In [ ]:
keys = {"a":0, "b":1, "c":2, "d":3}

In [ ]:

train_datas = {k: datas[keys[k]][:int(data.shape[0]*0.8)] for k in keys}
train_datas["a"].shape

In [ ]:
test_datas = {k: datas[keys[k]][int(data.shape[0]*0.8):] for k in keys}
test_datas["a"].shape

In [ ]:
test_datas["b"].head()

In [ ]:
train_datas["a"].head()

In [ ]:
train_datas["b"].head()

In [ ]:
train_datas["c"].head()

In [ ]:
input_dims = {k: (np.array(train_datas[k]).astype(float))[0].shape[0] for k in train_datas}

# Was getting a str() datatype error without this line below
#train_d = (np.array(data).astype(float))[:int((np.array(data).astype(float)).shape[0]*0.8)]

print(input_dims)

Encoder

In [ ]:
mixer = Mixer()

In [ ]:
autoencoder_set = mixer.make_new({'a': model_shape, 'b': model_shape, 'c': model_shape, 'd': model_shape}, latent_dim, input_dims, ["a", "b", "c", "d"])

In [ ]:
# new = Autoencoder_set(encoders, decoders, latent_dim, input_dims)
# new.copy(autoencoder_set)
# autoencoder_set = new

In [ ]:
track_num = 243
print([train_datas[d].values[track_num] for d in train_datas])

In [ ]:
#ignore = [(a, b) for a in range(num_sets) for b in range(num_sets) if (a != (b+1)%num_sets)]
#ignore2 = []#[(a, a) for a in range(num_sets)]
#autoencoder_set.train_set_old(train_datas, 5, batch_size = 64, ignore_pairs = ignore, track_num = track_num)
settings = {
    "training": [["$all", "$all", True, True]],#[[[k1], [k2 for k2 in keys if k1 != k2], True, True] for k1 in keys],
    #"encoder_proximity_training": [["$all", True]],
    "plot": [True, 3, [0, 1, 2]]
    }
autoencoder_set.train_set(train_datas, 25, autoencoder_set.make_train_config(settings = settings), batch_size = 64, track_num = track_num)

In [ ]:
cold = {x: list(train_datas[x]) for x in train_datas}
mixer.save_to_label(autoencoder_set, extra = {"columns" : cold}, label = label)
#autoencoder_set.save_models(label = label, extra = {"columns" : cold})

In [ ]:
# extra = autoencoder_set.load_models(label = label)
# train_datas, test_datas = make_train_test(split_by_col_l(data, [list(extra["columns"][str(x)]) for x in range(num_sets)]), 0.8)

In [ ]:
autoencoder_set.eval_set(test_datas)

In [ ]:
autoencoder_set.show_accuracy(test_datas)

In [ ]:
#autoencoder_set.show_error_distribution(test_datas)

In [ ]:
autoencoder_set.show_error_accuracy(test_datas)

In [ ]:
autoencoder_set.show_binary_accuracy(test_datas)

In [ ]:
scatter_train = autoencoder_set.plot_to_latent_space([t.values for t in train_datas])
scatter_test = autoencoder_set.plot_to_latent_space([t.values for t in test_datas])

In [ ]:
if latent_dim == 2:
    mapqL = autoencoder_set.map_latent_space(100)

train scatter

In [ ]:
autoencoder_set.make_scatter(scatter_train[0], 3, [0, 1, 2])
autoencoder_set.make_scatter(scatter_train[1], 3, [0, 1, 2])
autoencoder_set.make_scatter(scatter_train[2], 3, [0, 1, 2])
autoencoder_set.make_scatter(np.concatenate(scatter_train), 3, [0, 1, 2])

In [ ]:
sub = 243

item = np.concatenate((scatter_train[0, sub:sub+1],scatter_train[1, sub:sub+1],scatter_train[2, sub:sub+1]))
print(item)
autoencoder_set.make_scatter(item, 3, [0, 1, 2])

test scatter

In [ ]:
autoencoder_set.make_scatter(scatter_test[0], 3, [0, 1, 2])
autoencoder_set.make_scatter(scatter_test[1], 3, [0, 1, 2])
autoencoder_set.make_scatter(scatter_test[2], 3, [0, 1, 2])
autoencoder_set.make_scatter(np.concatenate(scatter_test), 3, [0, 1, 2])

In [ ]:
sub = 432

item = np.concatenate(([s[sub:sub+1] for s in scatter_test]))
print(item)
autoencoder_set.make_scatter(item, 3, [0, 1, 2])

map

In [ ]:
if latent_dim == 2:
    
    for i, dim in enumerate(input_dims):
        print(i, dim)
        for qi in range(dim):
            plt.imshow(mapqL[i][qi])
            plt.colorbar()
            plt.pause(0.01)
    plt.show()

Test/demo?

In [ ]:
# e = 0
# Qlist = list(train_datas[e])
# print(Qlist)
# resp = interface(Qlist)[list(train_datas[e])]

In [ ]:
# enc = autoencoder_set.make_encoding(resp, e)
# plt.imshow([[enc[0]]])

In [ ]:
# d = 1
# pred = autoencoder_set.make_prediction(resp, e, d)
# present(list(train_datas[d]), pred)